In [19]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from google.cloud import storage
from shapely.wkt import loads
import itertools

In [81]:
# Crear una instancia del cliente BigQuery usando el archivo de credenciales JSON
credentials_path = 'C:/Users/NoxiePC/Desktop/henry/driven-atrium-445021-m2-a773215c2f46.json'
credentials = service_account.Credentials.from_service_account_file(credentials_path)

# Crear el cliente BigQuery con las credenciales
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

# Definir la consulta SQL para acceder a la vista
query_viajes_por_ubicacion_y_tiempoPU = """SELECT * FROM `driven-atrium-445021-m2.project_data.viajes_por_ubicacion_y_tiempo` LIMIT 1000;"""
query_viajes_por_ubicacion_y_tiempoDO = """SELECT * FROM `driven-atrium-445021-m2.project_data.oferta_por_ubicacion` LIMIT 1000;"""
query_weather_borough_details = """SELECT * FROM `driven-atrium-445021-m2.project_data.weather_borough_details` LIMIT 1000;"""
query_coordinates = """SELECT * FROM driven-atrium-445021-m2.project_data.coordinates"""

# Ejecutar la consulta y cargar los resultados en un DataFrame de pandas
df_coordinates = client.query(query_coordinates).to_dataframe()
df_clima = client.query(query_weather_borough_details).to_dataframe()
df_viajesPU = client.query(query_viajes_por_ubicacion_y_tiempoPU).to_dataframe()
df_viajesDO = client.query(query_viajes_por_ubicacion_y_tiempoDO).to_dataframe()


c:\Users\NoxiePC\anaconda3\Lib\site-packages\google\cloud\bigquery\table.py:1785: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
c:\Users\NoxiePC\anaconda3\Lib\site-packages\google\cloud\bigquery\table.py:1785: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
c:\Users\NoxiePC\anaconda3\Lib\site-packages\google\cloud\bigquery\table.py:1785: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
c:\Users\NoxiePC\anaconda3\Lib\site-packages\google\cloud\bigquery\table.py:1785: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [107]:
clima = df_clima
viajesPU = df_viajesPU
viajesDO = df_viajesDO

In [108]:
# DEJAMOS LISTA EL DATASET DE COORDENADAS  
coordinates = df_coordinates[~df_coordinates['location_id'].isin([264, 265, 1])]
boroughs = ["Queens", "Brooklyn", "Manhattan", "Bronx", "Staten Island"]
coordinates = coordinates[coordinates['borough'].isin(boroughs)]
# Extraer coordenadas centrales (centroides) de las zonas
coordinates['geometry'] = coordinates['geom'].apply(loads)  # Convertir a geometría
coordinates['lat'] = coordinates['geometry'].apply(lambda x: x.centroid.y)
coordinates['lon'] = coordinates['geometry'].apply(lambda x: x.centroid.x)

coordinates = coordinates[['geometry', 'lat', 'lon', 'location_id', 'borough']]
coordinates = coordinates.rename(columns={'location_id':'locationID'})
coordinates


,geometry,lat,lon,locationID,borough
1,MULTIPOLYGON (((-73.84792614099985 40.87134223...,40.864474,-73.847422,3,Bronx
2,MULTIPOLYGON (((-73.88513907699999 40.86638287...,40.867682,-73.890183,18,Bronx
3,MULTIPOLYGON (((-73.88386792099986 40.86429088...,40.857780,-73.885867,20,Bronx
4,MULTIPOLYGON (((-73.87094462199981 40.85726608...,40.857747,-73.875476,31,Bronx
5,MULTIPOLYGON (((-73.85882515999995 40.85806135...,40.864002,-73.864901,32,Bronx
...,...,...,...,...,...
258,MULTIPOLYGON (((-74.08221272914942 40.64828016...,40.638972,-74.102313,206,Staten Island
259,MULTIPOLYGON (((-74.08491019899996 40.59570472...,40.586787,-74.085512,214,Staten Island
260,MULTIPOLYGON (((-74.07530550099992 40.63772995...,40.618769,-74.073704,221,Staten Island
261,MULTIPOLYGON (((-74.09787969199995 40.64035805...,40.630049,-74.102860,245,Staten Island


In [119]:
# CREAMOS EL DATASET AUXILIAR 
locaciones = coordinates['locationID'].unique()

clima = clima.rename(columns={'day_of_month':'day', 'hour_of_day':'hour'})
df = pd.DataFrame()
df['datetime'] = pd.to_datetime(clima[['year', 'month', 'day', 'hour']])
date_range = pd.date_range(start=df['datetime'].min(), end=df['datetime'].max(), freq='h')
df_aux = pd.DataFrame(date_range, columns=['datetime'])
clima = clima.rename(columns={'day':'day_of_month', 'hour':'hour_of_day'})

In [124]:
df_aux = list(itertools.product(df_aux['datetime'], locaciones))
df_aux = pd.DataFrame(df_aux, columns=['datetime', 'locationID'])

In [125]:
df_aux['year'] = df_aux['datetime'].dt.year
df_aux['month'] = df_aux['datetime'].dt.month
df_aux['day_of_month'] = df_aux['datetime'].dt.day
df_aux['hour_of_day'] = df_aux['datetime'].dt.hour
df_aux['day_of_week'] = df_aux['datetime'].dt.dayofweek
df_aux = df_aux.drop(columns=['datetime'])

In [143]:
viajesDO

,locationID,year,month,day_of_month,day_of_week,hour_of_day,oferta
0,185,2024,3,18,1,11,91
1,213,2024,3,18,1,23,127
2,51,2024,3,4,1,11,107
3,32,2024,3,4,1,16,75
4,159,2024,3,4,1,20,114
...,...,...,...,...,...,...,...
995,168,2024,3,19,2,6,134
996,48,2024,3,19,2,23,485
997,231,2024,3,5,2,9,745
998,238,2024,3,12,2,20,376


In [144]:
resultado = pd.DataFrame()

resultado = df_aux.merge(viajesDO, on=['locationID', 'year', 'month', 'day_of_month', 'hour_of_day', 'day_of_week'], how='left')
resultado['oferta'] = resultado['oferta'].fillna(0)
resultado = resultado.merge(viajesPU, on=['locationID', 'year', 'month', 'day_of_month', 'hour_of_day', 'day_of_week'], how='left')
resultado['solicitudes'] = resultado['solicitudes'].fillna(0)
resultado['promedio_fare_amount'] = resultado['promedio_fare_amount'].fillna(0)


In [146]:
trip_data = resultado

In [ ]:
# Supongamos que tu DataFrame se llama trip_data y tiene una columna 'datetime_old'
# Creamos la nueva columna 'datetime'
trip_data['datetime'] = pd.to_datetime(trip_data['datetime_old'])

# Filtramos el rango de fechas
date_range = pd.date_range(start=trip_data['datetime'].min(), end=trip_data['datetime'].max(), freq='h')
trip_data = trip_data[trip_data['datetime'].isin(date_range)]

# Eliminamos la columna original 'datetime_old'
trip_data = trip_data.drop(columns=['datetime_old'])

# Opcional: reordenamos las columnas si es necesario
trip_data = trip_data[['locationID', 'year', 'month', 'day_of_month', 'hour_of_day', 'day_of_week', 'oferta', 'solicitudes', 'promedio_fare_amount']]


KeyError: 'datetime'

In [145]:

#  # Realizar el merge para agregar la columna 'Borough' al dataset 'trip_data' 
trip_data = trip_data.merge(coordinates[['locationID', 'borough']], on='locationID', how='left')
trip_data = trip_data.merge(clima, on=['year', 'month', 'day_of_month', 'hour_of_day', 'day_of_week', 'borough'], how='left')

trip_data

,locationID,year,month,day_of_month,hour_of_day,day_of_week,oferta,solicitudes,promedio_fare_amount,borough,temperature,relative_humidity,apparent_temperature,weather_code,cloud_cover,wind_speed,wind_gusts
0,3,2023,1,2,3,0,0,0,0.0,Bronx,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,18,2023,1,2,3,0,0,0,0.0,Bronx,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20,2023,1,2,3,0,0,0,0.0,Bronx,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,31,2023,1,2,3,0,0,0,0.0,Bronx,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,32,2023,1,2,3,0,0,0,0.0,Bronx,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4187017,206,2024,10,28,23,0,0,0,0.0,Staten Island,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4187018,214,2024,10,28,23,0,0,0,0.0,Staten Island,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4187019,221,2024,10,28,23,0,0,0,0.0,Staten Island,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4187020,245,2024,10,28,23,0,0,0,0.0,Staten Island,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [140]:
trip_data.isnull().sum()

locationID                    0
year                          0
month                         0
day_of_month                  0
hour_of_day                   0
day_of_week                   0
solicitudes                   0
promedio_fare_amount          0
borough                       0
temperature             4133240
relative_humidity       4133240
apparent_temperature    4133240
weather_code            4133240
cloud_cover             4133240
wind_speed              4133240
wind_gusts              4133240
dtype: int64

In [117]:
# Variables predictoras (X) y variable objetivo (y)
predictors = [
    'locationID', 'day_of_month','hour_of_day', 'day_of_week',
    'relative_humidity', 'apparent_temperature','temperature', 'weather_code',
    'cloud_cover','wind_speed', 'wind_gusts'
]

X = trip_data[predictors]
y1 = trip_data['solicitudes']
y2 = trip_data['oferta']
y3 = trip_data['promedio_fare_amount']

# Dividir los datos en conjunto de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y1_train, y1_test = train_test_split(X, y1, test_size=0.2, random_state=42)
X_train, X_test, y2_train, y2_test = train_test_split(X, y2, test_size=0.2, random_state=42)
X_train, X_test, y3_train, y3_test = train_test_split(X, y3, test_size=0.2, random_state=42)

# Crear el modelo XGBoost
model_1 = xgb.XGBRegressor(
    n_estimators=80,
    max_depth=15,
    learning_rate=0.2,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
model_2 = xgb.XGBRegressor(
    n_estimators=80,
    max_depth=15,
    learning_rate=0.2,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
model_3 = xgb.XGBRegressor(
    n_estimators=80,
    max_depth=15,
    learning_rate=0.2,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

# Entrenar el modelo con los datos de entrenamiento
model_1.fit(X_train, y1_train)
model_2.fit(X_train, y2_train)
model_3.fit(X_train, y3_train)


KeyError: 'oferta'